In [1]:
import json
import os
import itertools
from typing import Tuple

import pandas as pd
import numpy as np
import talib as ta
from tqdm import tqdm

from ftx_client import FtxClient
import supertrend as spt
import backtesting as bt
from config import API_KEY, API_SECRET

In [2]:
with open("settings.json") as jsonfile:
    settings = json.load(jsonfile)
    
BACKTEST_FOLDER = settings["filepaths"]["backtest_folder"]
OPTIMIZEDML_FILEPATH = os.path.join(BACKTEST_FOLDER, settings["filepaths"]["optimized_ml_file"])
ANALYSIS_FILEPATH = os.path.join(BACKTEST_FOLDER, settings["filepaths"]["analysis_file"])

In [3]:
ftx = FtxClient(api_key=API_KEY, api_secret=API_SECRET)

In [4]:
markets = []
for future in ftx.list_futures():
    if future["type"] == "perpetual":
        if (future["volumeUsd24h"] > settings["markets"]["min_volume_usd_24h"] and 
                future["name"] not in settings["markets"]["blacklist"]):
            markets.append(future["name"])

In [5]:
optimzed_ml = pd.read_csv(OPTIMIZEDML_FILEPATH)

In [6]:
analysis_df = pd.DataFrame(columns=["Name", "AvgReturns", "StdDev", "RetDevRatio", "MinReturns", "MaxReturns",
             "AvgNegReturns", "AvgPosReturns", "PosNegRetRatio", "MedReturns", "MedNegReturns",
             "MedPosReturns", "MedPosNegRetRatio"])

pbar = tqdm(markets)
for market in pbar:
    pbar.set_description(market)
    df = ftx.get_historical_market_data(market, interval=settings["analysis"]["interval"], start_time=settings["analysis"]["start_time"])
    
    if len(df) < settings["analysis"]["min_data_length"]:
        continue
        
    multiplier = optimzed_ml.loc[optimzed_ml['Name'] == market]["Multiplier"].values[0]
    lookback = optimzed_ml.loc[optimzed_ml['Name'] == market]["Lookback"].values[0]
    
    profits = bt.backtest_dataframe(df, look_back=lookback, multiplier=multiplier)
    profits["Name"] = market
    analysis_df = analysis_df.append(profits, ignore_index=True)

ZEC-PERP: 100%|██████████| 54/54 [02:46<00:00,  3.09s/it]  


In [7]:
analysis_df = analysis_df.round(1)
analysis_df.to_csv(ANALYSIS_FILEPATH, index=False)

In [8]:
pd.read_csv(ANALYSIS_FILEPATH)

,Name,AvgReturns,StdDev,RetDevRatio,MinReturns,MaxReturns,AvgNegReturns,AvgPosReturns,PosNegRetRatio,MedReturns,MedNegReturns,MedPosReturns,MedPosNegRetRatio
0,1INCH-PERP,19.4,26.0,0.7,-6.1,84.4,-5.1,26.4,5.2,9.9,-5.1,17.2,3.4
1,AAVE-PERP,7.2,18.5,0.4,-16.7,36.1,-10.2,21.6,2.1,6.7,-10.3,22.5,2.2
2,ADA-PERP,10.6,21.3,0.5,-13.7,65.0,-5.0,23.5,4.7,3.6,-2.2,15.8,7.3
3,ALGO-PERP,13.6,17.0,0.8,-2.6,44.9,-2.6,17.6,6.9,4.1,-2.6,11.0,4.3
4,ALPHA-PERP,72.4,130.0,0.6,-2.4,385.2,-2.3,102.3,44.6,26.9,-2.3,27.1,11.8
5,ALT-PERP,8.8,21.6,0.4,-14.9,71.7,-5.7,21.1,3.7,0.4,-4.2,17.7,4.2
6,AXS-PERP,42.6,129.5,0.3,-19.8,517.3,-10.3,89.0,8.6,11.8,-10.5,17.7,1.7
7,BCH-PERP,27.9,34.8,0.8,-1.7,108.7,-1.7,31.6,18.3,12.9,-1.7,17.8,10.3
8,BNB-PERP,10.1,28.4,0.4,-16.9,101.9,-6.1,24.0,3.9,2.7,-5.2,9.1,1.8
9,BSV-PERP,44.1,64.1,0.7,4.5,171.4,0.0,44.1,44.1,11.7,0.0,11.7,11.7
